In [2]:
import tabula
import pandas as pd

file_name = "eStatement_Feb2024.pdf"
output_file_name= file_name.split('.')[0]
 
dfs = tabula.read_pdf(file_name, pages='all', encoding='utf-8', guess=False, lattice=False, stream=True)

appended_data = []

for df in dfs[1:]:
    df3 = df.drop_duplicates();
    df4 = df3.dropna(axis=1, how='all');
    df5 = df4.dropna(axis=0, how='all');

    df6 = df5.map(lambda x: x.strip() if isinstance(x, str) else x)

    df6.columns = [*df6.columns[:-1], 'Amount']

    df6['Date']=df5['Unnamed: 0'].str.slice(stop=6)+' 2023'
    df6['Description']=df5['Unnamed: 0'].str.slice(start=6)
    df7 = df6.filter(['Date','Description','Amount'])

    df7['Date'] = pd.to_datetime(df7['Date'],format='%d %b %Y',errors='coerce')
    df7 = df7.dropna(subset=['Date'])
    df7['Amount'] = df7['Amount'].astype(str)
    df7['Amount'] = df7['Amount'].str.replace(r'[^0-9.]', '', regex=True)
    df7['Amount'] = pd.to_numeric(df7['Amount'], errors='coerce')

    appended_data.append(df7)
 
appended_data = pd.concat(appended_data).reset_index(drop=True)

grouped_df = appended_data.groupby(['Description']).agg({'Date':'min','Amount': 'sum'}).reset_index().sort_values(by='Amount',ascending=False)

desired_column_order = ['Date','Description','Amount']  

reodered_df = grouped_df[desired_column_order]
reodered_df.to_csv(output_file_name+'.csv',index=False)